In [ ]:
pip install keras==2.2.4 tensorflow==2.3.0 coremltools==5.1.0 pillow==7.0.0 h5py==2.10.0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
%matplotlib inline

In [ ]:
(x_train, y_train), (x_test, y_test)  = mnist.load_data()

In [ ]:
import sys

print('以下の数字は' + str(y_train[0]) + 'です')
for item in x_train[0]:
  for i in item:
    sys.stdout.write(str(i).zfill(3))
  sys.stdout.write('\n')

In [ ]:
x_train = x_train.reshape(60000, 28, 28, 1).astype("float32") / 255
x_test = x_test.reshape(10000, 28, 28, 1).astype("float32") / 255

print(x_train[0].shape)
print(x_train[0])

In [ ]:
y_train = to_categorical(y_train.astype("float32"), 10)
y_test = to_categorical(y_test.astype("float32"), 10)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D

model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='sgd',
    metrics=['acc']
    )

In [ ]:
model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=30
                    )

In [ ]:
#モデルの検証

from PIL import Image, ImageFilter

im_file = 'train_2.png'
# im_file = 'train_5.jpeg'

im = Image.open(im_file)
im = im.convert('L') # グレースケール

im = im.resize((28, 28)) # サイズ調整
im = np.array(im)
im = im.astype('float32') # intからfloatに
im = 255 - im #値反転 # im = ImageOps.invert(im) # 値反転
im = im[np.newaxis, ...]
im = im[..., np.newaxis]
print(im.shape)

for item in im:
  for i in item:
    sys.stdout.write(str(i).zfill(3))
  sys.stdout.write('\n')

# 画像ファイル変換

# im = np.array(im)
# im = im.reshape(28, 28)

In [ ]:
predictions_single = model.predict(im)

In [ ]:
print(predictions_single)
print(predictions_single[0].argmax())

In [ ]:
model.save('./image_classification.h5')

In [ ]:
import coremltools as ct

image_labels = [
   '0',
   '1',
   '2',
   '3',
   '4',
   '5',
   '6',
   '7',
   '8',
   '9'
]

classifier_config = ct.ClassifierConfig(image_labels)
image_input = ct.ImageType(shape=(1, 28, 28, 1,), scale=1/255)

mlmodel = ct.convert("image_classification.h5",  
                     inputs=[image_input],
                     classifier_config=classifier_config
                     )

mlmodel.save('h5_model.mlmodel')

In [ ]:
import coremltools

coreml_model_path = 'h5_model.mlmodel'
spec = coremltools.utils.load_spec(coreml_model_path)
builder = coremltools.models.neural_network.NeuralNetworkBuilder(spec=spec)
builder.inspect_input_features()